## Decide Number of Neurons and hidden layer

In [1]:
## pip install keras-tuner --upgrade


In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import RandomSearch

D:\Anaconda\envs\gpu\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


In [3]:
df = pd.read_csv('../..//Datasets/Real_Combine.csv')
## air quality index feature dataset
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [28]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

How many neurons and all in the architecture??

How many hidden layers and all in the architecture??

Learning Rate parameter

In [40]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)): ## hidden layers ranging from 2 to 20
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32,max_value=512,step=32),activation='relu'))
        ## minimum neurons is 32, max is 512 at step size of 32

        model.add(layers.Dense(1,activation='linear'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
                loss='mean_absolute_error',
                metrics=['mean_absolute_error'])
    
    return model


In [41]:
tuner = RandomSearch(
        build_model,
        objective='val_mean_absolute_error',
        max_trials=5,
        executions_per_trial=3,
        directory='project',
        project_name='Air_Quality_Index')

## total it will run 15 times: 5*3        

In [42]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [46]:
import numpy as np
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test =  np.asarray(X_test)
y_test = np.asarray(y_test)

In [47]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))


Trial 5 Complete [00h 00m 07s]
val_mean_absolute_error: 50.63099670410156

Best val_mean_absolute_error So Far: 50.63099670410156
Total elapsed time: 00h 02m 12s
INFO:tensorflow:Oracle triggered exit


In [49]:
tuner.results_summary()

## see which is giving the best score

Results summary
Results in project\Air_Quality_Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 4
units_0: 224
learning_rate: 0.01
units_1: 384
units_2: 128
units_3: 384
units_4: 64
units_5: 32
units_6: 160
units_7: 480
units_8: 352
units_9: 192
units_10: 352
units_11: 352
units_12: 480
units_13: 512
units_14: 192
units_15: 32
units_16: 416
units_17: 224
Score: 50.63099670410156
Trial summary
Hyperparameters:
num_layers: 16
units_0: 480
learning_rate: 0.01
units_1: 384
units_2: 384
units_3: 288
units_4: 160
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
Score: 59.45698165893555
Trial summary
Hyperparameters:
num_layers: 7
units_0: 288
learning_rate: 0.001
units_1: 448
units_2: 160
units_3: 128
units_4: 288
units_5: 224
units_6: 224
units_7: 384
units_8: 128
units_9: 448
units_10: 512
units_11: 320
units_12: 128
units_13: 288
units_14: 224
units_15: 256
Score: 63.73872756958